In [10]:
import os
import csv
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from github import Github

In [11]:
# Load environment variables
auth_token = os.getenv("GITHUB_TOKEN")
username = os.getenv("GITHUB_USERNAME")

# Using an access token
g = Github(auth_token)
 
# Get the repositories
repos = g.get_repos()
users = []

In [12]:
num_repos = 1000
repos_to_shuffle = 10000
repos = list(repos[:repos_to_shuffle])

# To repeat the same random selection we can fix the seed
np.random.seed(42)
repos = np.random.choice(repos, num_repos, replace=False)

repos_dataset = []
# repos = ["r_"+str(i)]

In [13]:
users = []
for i, repo in enumerate(repos):
    # s = repo.get_subscribers()
    try:
        r_id = "r_" + str(i)
        languages = list(repo.get_languages().keys())
        full_name = repo.full_name
        repos_dataset.append({'id': r_id, 'name': full_name, 'languages': languages})
        
        # contributors
        cs = repo.get_contributors()
        cs = [{'repos': r_id, 'user': user.login} for user in cs]        
        users.extend(list(cs))
        
        # subscribers
        # subs = repo.get_subscribers()
        # subs = [{'repos': r_id, 'user': user.login} for user in subs]        
        # users.extend(list(subs))
    except Exception as e:
        print('Error for repo: ', repo.full_name, e)
        continue

Error for repo:  nowa/chit 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-languages"}
Error for repo:  etrangedev/desert_tools 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-languages"}
Error for repo:  stephan/mymimir 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-languages"}
Error for repo:  gaskett/spree 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-languages"}
Error for repo:  kowal/daldosso 403 {"message": "Repository access blocked", "block": {"reason": "unavailable", "created_at": "2014-04-01T08:23:50Z", "html_url": "https://github.com/tos"}}


In [ ]:
# reduce the users by username
users_df = pd.DataFrame(users)

users_df = users_df.groupby('user')['repos'].apply(list).reset_index(name='repos')
users_id = ["u_"+str(i) for i in range(len(users_df))]
users_df['id'] = users_id
users_df = users_df[['id', 'user', 'repos']]

users_df.to_csv('dataset/users.csv', index=False)

In [ ]:
pd.DataFrame(repos_dataset).to_csv('dataset/repos.csv', index=False)